In [1]:

import torch
import pandas as pd
import numpy as np
from torch.utils.data import DataLoader, TensorDataset
import gpytorch
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import gc

gc.collect()
torch.cuda.empty_cache()


In [2]:



def load_data(train_file):
    col_names = ['unit', 'cycle'] + \
                [f'op_setting_{i}' for i in range(1, 4)] + \
                [f'sensor_{i}' for i in range(1, 22)]
    df = pd.read_csv(train_file, delim_whitespace=True, header=None, names=col_names)
    max_cycle = df.groupby('unit')['cycle'].max().reset_index()
    max_cycle.columns = ['unit', 'max_cycle']
    df = df.merge(max_cycle, on='unit', how='left')
    df['RUL'] = df['max_cycle'] - df['cycle']
    return df

# 2) Preprocessing (features + scaled target + raw RUL)
def preprocess(df, unit_ids=None):
    if unit_ids is not None:
        df = df[df['unit'].isin(unit_ids)]
    raw_rul = df['RUL'].values  # save true RUL
    
    df = df.drop(columns=['unit', 'cycle', 'max_cycle', 'RUL'])
    # Normalize features to [-1, 1] range
    
    X = df.values
    y = raw_rul.reshape(-1, 1)
    fscaler = MinMaxScaler(feature_range=(-1, 1))
    tscaler = MinMaxScaler(feature_range=(-1, 1))
    X = fscaler.fit_transform(X)
    # y = tscaler.fit_transform(y)
    X = torch.from_numpy(X).float()
    y = torch.from_numpy(y).flatten().float()
    return X, y, raw_rul, fscaler, tscaler

#  Kernel factory
def get_kernel(kernel_type, **kwargs):
 
 
# Base kernels
    if kernel_type == 'RBF':
        base = gpytorch.kernels.RBFKernel(**kwargs)
    elif kernel_type == 'Matern':
        base = gpytorch.kernels.MaternKernel(**kwargs)  # default nu=2.5
    elif kernel_type == 'Mat32':
        base = gpytorch.kernels.MaternKernel(nu=1.5, **kwargs)
    elif kernel_type == 'Mat52':
        base = gpytorch.kernels.MaternKernel(nu=2.5, **kwargs)
    elif kernel_type == 'Linear':
        base = gpytorch.kernels.LinearKernel(**kwargs)
    elif kernel_type == 'Periodic':
        base = gpytorch.kernels.PeriodicKernel(**kwargs)
    elif kernel_type == 'RQ':
        base = gpytorch.kernels.RQKernel(**kwargs)
    elif kernel_type == 'Poly':
        base = gpytorch.kernels.PolynomialKernel(**kwargs)
    elif kernel_type == 'SpectralMixture':
        # e.g. num_mixtures=4, ard_num_dims=X.shape[1]
        nm = kwargs.pop('num_mixtures', 4)
        base = gpytorch.kernels.SpectralMixtureKernel(num_mixtures=nm, **kwargs)

    # Composite kernels
    elif kernel_type == 'RBF_Linear':
        base = gpytorch.kernels.RBFKernel(**kwargs) + gpytorch.kernels.LinearKernel(**kwargs)

    elif kernel_type == 'Matern_Linear':
        base = gpytorch.kernels.MaternKernel(**kwargs) + gpytorch.kernels.LinearKernel(**kwargs)
    elif kernel_type == 'Mat52_Linear':
        base = (
            gpytorch.kernels.MaternKernel(nu=2.5, **kwargs)
            + gpytorch.kernels.LinearKernel(**kwargs)
        )
    elif kernel_type == 'RQ':
        base = gpytorch.kernels.RQKernel(**kwargs)
    elif kernel_type == 'RQ_Linear':
        base = gpytorch.kernels.RQKernel(**kwargs) + gpytorch.kernels.LinearKernel(**kwargs)
    elif kernel_type == 'Periodic_RBF':
        base = gpytorch.kernels.PeriodicKernel(**kwargs) + gpytorch.kernels.RBFKernel(**kwargs)
    else:
        raise ValueError(f"Unknown kernel: {kernel_type}")

    # Wrap in a ScaleKernel so we learn an output-scale
    return gpytorch.kernels.ScaleKernel(base)



# Now preprocess just this single-sensor dataset
def preprocess_single_sensor(df, sensor_col, unit_ids=None):

    raw_rul = df['RUL'].values
    X = df[[sensor_col]].values
    y = raw_rul.reshape(-1, 1)
    fscaler = MinMaxScaler(feature_range=(-1, 1))
    tscaler = MinMaxScaler(feature_range=(-1, 1))
    X_scaled = fscaler.fit_transform(X)
    # y_scaled = tscaler.fit_transform(y)
    return (
        torch.from_numpy(X_scaled).float(),
        torch.from_numpy(y).flatten().float(),
        raw_rul,
        fscaler,
        tscaler
    )
    
    
# Test train split function
def train_test_split(X, y, test_size=0.2, random_state=None):
    if random_state is not None:
        torch.manual_seed(random_state)
    indices = torch.randperm(X.size(0))
    split_idx = int(X.size(0) * (1 - test_size))
    train_indices = indices[:split_idx]
    test_indices = indices[split_idx:]
    
    train_x = X[train_indices]
    train_y = y[train_indices]
    test_x = X[test_indices]
    test_y = y[test_indices]
    
    return train_x, train_y, test_x, test_y
    
    
    
    
#_________________________GP Functions_______________________
class ExactGPModel(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood, kernel):
        super(ExactGPModel, self).__init__(train_x, train_y, likelihood)
        self.mean_module = gpytorch.means.ConstantMean()
        self.covar_module = kernel

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)


# def train_model(model, likelihood, train_x, train_y, num_epochs=50):
#     # Find optimal model hyperparameters
#     model.train()
#     likelihood.train()

#     # Use the adam optimizer
#     optimizer = torch.optim.Adam(model.parameters(), lr=0.1)  # Includes GaussianLikelihood parameters

#     # "Loss" for GPs - the marginal log likelihood
#     mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)

#     # training_iter = 50
#     for i in range(num_epochs):
#         # Zero gradients from previous iteration
#         optimizer.zero_grad()
#         # Output from model
#         output = model(train_x)
#         # Calc loss and backprop gradients
#         loss = -mll(output, train_y)
#         loss.backward()
#         print('Iter %d/%d - Loss: %.3f   lengthscale: %.3f   noise: %.3f' % (
#             i + 1, num_epochs, loss.item(),
#             model.covar_module.base_kernel.lengthscale.item(),
#             model.likelihood.noise.item()
#         ))
#         optimizer.step()
        
#     return model, likelihood

# 

In [3]:


def train_model(model, likelihood, train_x, train_y, num_epochs=50):
    loss = []
    model.train()
    likelihood.train()

    optimizer = torch.optim.Adam(model.parameters(), lr=0.1)
    mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)

    for epoch in range(1, num_epochs + 1):
        optimizer.zero_grad()
        output = model(train_x)
        loss = -mll(output, train_y)
        loss.backward()
        optimizer.step()

        print(f"Iter {epoch}/{num_epochs} - Loss: {loss.item():.3f}")
    return model, likelihood

# Test the model with the test set
def test_model(model, likelihood, test_x, test_y):
    model.eval()
    likelihood.eval()

    with torch.no_grad(), gpytorch.settings.fast_pred_var():
        observed_pred = likelihood(model(test_x))

    # Get the mean and variance of the predictions
    pred_mean = observed_pred.mean
    pred_var = observed_pred.variance

    # Calculate RMSE
    rmse = torch.sqrt(torch.mean((pred_mean - test_y) ** 2)).item()
    mean_absolute_error = torch.mean(torch.abs(pred_mean - test_y)).item()
    
    return pred_mean.cpu().numpy(), pred_var.cpu().numpy(), rmse, mean_absolute_error


In [4]:
# Load in the data
train_file     = "train_FD001.csv"  # PHM08 train file
kernel_chosen = 'RBF'  # Choose your kernel here

# Load data
df = load_data(train_file)

# Drop bad sensors
drop_sensors = [1, 5, 6, 10, 16, 18, 19]
drop_cols = [f"op_setting_{i+1}" for i in range(3) ] + [f"sensor_{sensor}"for sensor in drop_sensors]
print(f"Dropping columns: {drop_cols}")
df.drop(drop_cols, axis=1, inplace=True)

# Preprocess the data
sensors_normalized, RuL, _, _, _ = preprocess(df)

# Split into train and test sets
train_x, train_y, test_x, test_y = train_test_split(sensors_normalized, RuL, test_size=0.2, random_state=42)

# data is ready to go

Dropping columns: ['op_setting_1', 'op_setting_2', 'op_setting_3', 'sensor_1', 'sensor_5', 'sensor_6', 'sensor_10', 'sensor_16', 'sensor_18', 'sensor_19']


C:\Users\lindy\AppData\Local\Temp\ipykernel_24940\3132579775.py:5: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv(train_file, delim_whitespace=True, header=None, names=col_names)


In [5]:
    
# Move  data to GPU
if torch.cuda.is_available():
    train_x = train_x.cuda()
    train_y = train_y.cuda()
    test_x  = test_x.cuda()
    test_y  = test_y.cuda()
    print("Data Moved to GPU")



kernel_types = ['RBF', 'Matern', 'Linear', 'RBF_Linear']#, 'Periodic', 'Matern_Linear', 'Mat52_Linear', 'RQ','RQ_Linear', 'Periodic_RBF']

# Move the model to GPU
likelihood = gpytorch.likelihoods.GaussianLikelihood().cuda()

results = []
for kernel_type in kernel_types:
    print(f"Training with kernel: {kernel_type}")
    kernel     = get_kernel(kernel_type= kernel_type).cuda()
    model      = ExactGPModel(train_x, train_y, likelihood, kernel).cuda()

    # Train
    model, likelihood = train_model(model, likelihood, train_x, train_y, num_epochs=100)

    # Test the model
    pred_mean, pred_var, rmse, mean_abs_error = test_model(model, likelihood, test_x, test_y)
    print(f"Test RMSE: {rmse:.4f}")
    print(f"Test Mean Absolute Error: {mean_abs_error:.4f}")
    
    # Save the model and metrics in a dictionary
    model_metrics = {
        'kernel': kernel_type,
        'model': model.cpu(),
        'likelihood': likelihood.cpu(),
        'pred_mean': pred_mean,
        'pred_var': pred_var,
        'rmse': rmse,
        'mean_abs_error': mean_abs_error
    }
    
    # save dictionary to a list
    
    results.append(model_metrics)
    
    gc.collect()
    torch.cuda.empty_cache()




Data Moved to GPU
Training with kernel: RBF
Iter 1/100 - Loss: 1203.511
Iter 2/100 - Loss: 1115.328
Iter 3/100 - Loss: 1034.189
Iter 4/100 - Loss: 960.330
Iter 5/100 - Loss: 892.835
Iter 6/100 - Loss: 830.063
Iter 7/100 - Loss: 773.843
Iter 8/100 - Loss: 722.938
Iter 9/100 - Loss: 676.764
Iter 10/100 - Loss: 637.352
Iter 11/100 - Loss: 605.716
Iter 12/100 - Loss: 580.794
Iter 13/100 - Loss: 561.397
Iter 14/100 - Loss: 540.096
Iter 15/100 - Loss: 516.504
Iter 16/100 - Loss: 492.408
Iter 17/100 - Loss: 469.817
Iter 18/100 - Loss: 450.269
Iter 19/100 - Loss: 433.899
Iter 20/100 - Loss: 419.701
Iter 21/100 - Loss: 407.072
Iter 22/100 - Loss: 395.851
Iter 23/100 - Loss: 385.324
Iter 24/100 - Loss: 375.206
Iter 25/100 - Loss: 365.616
Iter 26/100 - Loss: 356.416
Iter 27/100 - Loss: 347.700
Iter 28/100 - Loss: 339.261
Iter 29/100 - Loss: 331.498
Iter 30/100 - Loss: 324.401
Iter 31/100 - Loss: 317.767
Iter 32/100 - Loss: 311.885
Iter 33/100 - Loss: 306.829
Iter 34/100 - Loss: 302.506
Iter 35/10

# Investiage Results

In [6]:
import torch, gc

gc.collect()
torch.cuda.empty_cache()


In [8]:
for i in range(len(kernel_types)):
    print(f"Kernel: {results[i]['kernel']}")
    print(f"RMSE: {results[i]['rmse']:.4f}")
    print(f"Mean Absolute Error: {results[i]['mean_abs_error']:.4f}")
    print(f"Predictions Mean: {results[i]['pred_mean'][:5]}")
    print(f"Predictions Variance: {results[i]['pred_var'][:5]}")
    print("\n")

Kernel: RBF
RMSE: 45.9151
Mean Absolute Error: 32.1825
Predictions Mean: [122.06146 115.45743 140.46083 128.84917 102.67618]
Predictions Variance: [9.694321  9.767435  9.0960865 8.135926  9.541519 ]


Kernel: Matern
RMSE: 43.0338
Mean Absolute Error: 30.1534
Predictions Mean: [140.82455 130.9536  148.05858 128.77681 117.52444]
Predictions Variance: [16.492058 16.626595 15.824485 14.787728 16.41446 ]


Kernel: Linear
RMSE: 45.0057
Mean Absolute Error: 34.2502
Predictions Mean: [165.05795 124.40299 162.52916 153.25366 127.68254]
Predictions Variance: [17.22464  17.218721 17.214495 17.210775 17.217068]


Kernel: RBF_Linear
RMSE: 43.0180
Mean Absolute Error: 31.6094
Predictions Mean: [154.1899  124.8269  152.50644 128.4646  122.1718 ]
Predictions Variance: [34.58886  34.61811  34.40497  33.859146 34.533417]


